# Scientific Computing: Abstraction Levels with Heat Diffusion

This notebook demonstrates different levels of abstraction in scientific computing using a **2D heat diffusion simulation** as our example. Heat diffusion is governed by:

$$\frac{\partial T}{\partial t} = \alpha \nabla^2 T$$

Where:
- T = temperature field
- t = time
- α = thermal diffusivity  
- ∇² = Laplacian operator

We'll implement this using **explicit finite difference** method and show how abstraction improves code organization, reusability, and maintainability in scientific computing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from typing import Protocol, Dict, Any, Callable
import time

# Set up plotting parameters
plt.rcParams['figure.figsize'] = (12, 4)
plt.rcParams['font.size'] = 10

## Level 0: Direct Implementation

**Characteristics:**
- Everything hardcoded in cells
- Manual loops and array operations
- Copy-paste for different scenarios
- No reusability or parameterization

**When to use:** Quick prototyping, exploring equations, one-off calculations

In [ ]:
# Level 0: Direct hardcoded implementation
# Grid parameters (hardcoded)
nx, ny = 50, 50
dx = 1.0
alpha = 0.1  # Thermal diffusivity
dt = 0.01    # Time step
steps = 200  # Number of time steps

# Initialize temperature field
T = np.zeros((nx, ny))
# Hot spot in center
T[20:30, 20:30] = 100.0

# Manual simulation with explicit loops
for step in range(steps):
    T_new = T.copy()
    
    # Manual finite difference calculation
    for i in range(1, nx-1):
        for j in range(1, ny-1):
            # 2D heat equation: explicit finite difference
            laplacian = (T[i+1,j] + T[i-1,j] + T[i,j+1] + T[i,j-1] - 4*T[i,j]) / (dx*dx)
            T_new[i,j] = T[i,j] + alpha * dt * laplacian
    
    # Hardcoded boundary conditions (zero temperature)
    T_new[0, :] = 0
    T_new[-1, :] = 0  
    T_new[:, 0] = 0
    T_new[:, -1] = 0
    
    T = T_new

# Direct plotting
plt.figure(figsize=(8, 6))
plt.imshow(T, cmap='hot', origin='lower')
plt.colorbar(label='Temperature')
plt.title('Level 0: Heat Diffusion after 200 steps')
plt.show()

print(f"Max temperature: {T.max():.2f}")
print(f"Total heat: {T.sum():.2f}")

**Level 0 Problems:**
- Need to copy-paste entire code for different parameters
- Hard to test individual components
- Difficult to modify boundary conditions or initial conditions
- No way to reuse the numerical method

## Level 1: Functional Decomposition

**Characteristics:**
- Break problem into logical functions
- Parameterize key variables
- Separate concerns (physics, boundaries, visualization)
- Enable testing and reuse

**When to use:** Research code, parameter studies, when you need flexibility

In [ ]:
# Level 1: Functional approach with clear separation of concerns

def heat_diffusion_step(T, alpha=0.1, dt=0.01, dx=1.0):
    """Perform one explicit finite difference step for 2D heat equation.

    Parameters:
    - T: temperature field (2D numpy array)
    - alpha: thermal diffusivity
    - dt: time step
    - dx: spatial grid spacing
    """
    nx, ny = T.shape
    T_new = T.copy()

    # Vectorized finite difference (more efficient than nested loops)
    laplacian = np.zeros_like(T)
    laplacian[1:-1, 1:-1] = (T[2:, 1:-1] + T[:-2, 1:-1] +
                            T[1:-1, 2:] + T[1:-1, :-2] - 4*T[1:-1, 1:-1]) / (dx*dx)

    T_new[1:-1, 1:-1] = T[1:-1, 1:-1] + alpha * dt * laplacian[1:-1, 1:-1]
    return T_new

def apply_boundary_conditions(T, bc_type='dirichlet', bc_value=0.0):
    """Apply boundary conditions to temperature field.

    Parameters:
    - T: temperature field (2D numpy array)
    - bc_type: 'dirichlet' or 'neumann'
    - bc_value: fixed temperature value for Dirichlet BC
    """
    if bc_type == 'dirichlet':
        T[0, :] = bc_value
        T[-1, :] = bc_value
        T[:, 0] = bc_value
        T[:, -1] = bc_value
    elif bc_type == 'neumann':
        # Zero flux (insulated boundaries)
        T[0, :] = T[1, :]
        T[-1, :] = T[-2, :]
        T[:, 0] = T[:, 1]
        T[:, -1] = T[:, -2]
    return T

def create_initial_condition(nx, ny, condition_type='hotspot', temperature=100.0):
    """Create different types of initial temperature distributions.

    Parameters:
    - nx, ny: grid dimensions
    - condition_type: 'hotspot', 'gradient', or 'ring'
    - temperature: maximum temperature value
    """
    T = np.zeros((nx, ny))

    if condition_type == 'hotspot':
        # Central hot region
        cx, cy = nx//2, ny//2
        size = min(nx, ny) // 5
        T[cx-size:cx+size, cy-size:cy+size] = temperature

    elif condition_type == 'gradient':
        # Linear temperature gradient
        T = np.linspace(0, temperature, nx).reshape(-1, 1) * np.ones((1, ny))

    elif condition_type == 'ring':
        # Ring-shaped hot region
        cx, cy = nx//2, ny//2
        y, x = np.ogrid[:nx, :ny]
        r = np.sqrt((x - cx)**2 + (y - cy)**2)
        T[(r > nx//4) & (r < nx//3)] = temperature

    return T

def run_diffusion_simulation(nx=50, ny=50, alpha=0.1, dt=0.01, dx=1.0, steps=200,
                           initial_type='hotspot', bc_type='dirichlet', n_snapshots=5):
    """Run complete heat diffusion simulation with specified parameters.

    Parameters:
    - nx, ny: grid dimensions
    - alpha: thermal diffusivity
    - dt: time step
    - dx: spatial grid spacing
    - steps: number of time steps
    - initial_type: type of initial condition
    - bc_type: boundary condition type
    - n_snapshots: number of snapshots to save
    """
    # Initialize
    T = create_initial_condition(nx, ny, initial_type)
    history = [T.copy()]  # Store initial condition

    # Calculate snapshot interval - ensure we get meaningful snapshots
    snapshot_interval = max(1, steps // (n_snapshots - 1))

    # Time evolution
    for step in range(steps):
        T = heat_diffusion_step(T, alpha=alpha, dt=dt, dx=dx)
        T = apply_boundary_conditions(T, bc_type=bc_type)

        # Store snapshots at regular intervals
        if step % snapshot_interval == 0 or step == steps - 1:
            history.append(T.copy())

    return T, history

def plot_evolution(history, titles=None, figsize_per_plot=4):
    """Plot temperature evolution over time.

    Parameters:
    - history: list of temperature arrays
    - titles: list of titles for each subplot
    - figsize_per_plot: width per subplot in inches
    """
    n_plots = len(history)
    fig, axes = plt.subplots(1, n_plots, figsize=(figsize_per_plot*n_plots, 3))

    # Handle single plot case
    if n_plots == 1:
        axes = [axes]

    # Get consistent color scale across all plots
    vmax = max(T.max() for T in history)
    vmin = min(T.min() for T in history)

    for i, T in enumerate(history):
        im = axes[i].imshow(T, cmap='hot', origin='lower', vmin=vmin, vmax=vmax)
        axes[i].set_title(titles[i] if titles else f'Snapshot {i}')
        axes[i].axis('off')

    # Add colorbar that spans all subplots
    fig.colorbar(im, ax=axes, shrink=0.8, label='Temperature')
    plt.tight_layout()
    plt.show()

In [ ]:
# Level 1: Now we can easily run different scenarios

# Scenario 1: Hotspot with Dirichlet boundaries
T1, hist1 = run_diffusion_simulation(nx=50, ny=50, alpha=0.1, dt=0.01, dx=1.0, steps=200, 
                                    initial_type='hotspot', bc_type='dirichlet', n_snapshots=4)
plot_evolution(hist1[:4], titles=['Initial', 'Step 50', 'Step 100', 'Step 150'])

# Scenario 2: Ring with insulated boundaries  
T2, hist2 = run_diffusion_simulation(nx=50, ny=50, alpha=0.1, dt=0.01, dx=1.0, steps=200,
                                    initial_type='ring', bc_type='neumann', n_snapshots=4)
plot_evolution(hist2[:4], titles=['Ring Initial', 'Ring Step 50', 'Ring Step 100', 'Ring Step 150'])

# Easy parameter study - compare different thermal diffusivities
alphas = [0.05, 0.1, 0.2]
final_temps = []

print("Parameter study results:")
for alpha in alphas:
    T_final, _ = run_diffusion_simulation(nx=30, ny=30, alpha=alpha, dt=0.01, 
                                        dx=1.0, steps=100, initial_type='hotspot')
    final_temps.append(T_final.max())
    print(f"α = {alpha:.2f}: Max temperature = {T_final.max():.2f}")

# Show how different time steps affect the solution
fig, axes = plt.subplots(1, 3, figsize=(12, 3))
dts = [0.005, 0.01, 0.02]

for i, dt in enumerate(dts):
    T_final, _ = run_diffusion_simulation(nx=40, ny=40, alpha=0.1, dt=dt, 
                                        dx=1.0, steps=200, initial_type='hotspot')
    im = axes[i].imshow(T_final, cmap='hot', origin='lower')
    axes[i].set_title(f'dt = {dt:.3f}\\nMax T = {T_final.max():.1f}')
    axes[i].axis('off')

plt.colorbar(im, ax=axes, shrink=0.8, label='Temperature')
plt.suptitle('Level 1: Time Step Sensitivity Study')
plt.tight_layout()
plt.show()

**Level 1 Benefits:**
- Easy to test individual functions
- Parameter studies become trivial
- Different boundary conditions and initial conditions
- Reusable components for other PDE solvers

## Level 2: Object-Oriented Design

**Characteristics:**
- Encapsulate state and behavior in classes
- Method chaining for fluent interfaces
- Inheritance for different solver types
- Cleaner API for complex workflows

**When to use:** Complex simulations, multiple coupled equations, GUI applications

In [ ]:
# Level 2: Object-oriented approach with state management

class HeatGrid:
    """Represents the computational grid and temperature field."""
    
    def __init__(self, nx, ny, dx=1.0):
        self.nx, self.ny = nx, ny
        self.dx = dx
        self.T = np.zeros((nx, ny))
        self.history = []
        self.time = 0.0
        
    def set_initial_hotspot(self, center=None, size=None, temperature=100.0):
        """Set initial hotspot condition."""
        cx = center[0] if center else self.nx // 2
        cy = center[1] if center else self.ny // 2
        sz = size if size else min(self.nx, self.ny) // 5
        
        self.T[max(0, cx-sz):min(self.nx, cx+sz), 
               max(0, cy-sz):min(self.ny, cy+sz)] = temperature
        return self  # Method chaining
        
    def set_initial_gradient(self, direction='x', temp_range=(0, 100)):
        """Set linear temperature gradient."""
        if direction == 'x':
            temps = np.linspace(temp_range[0], temp_range[1], self.nx)
            self.T = temps.reshape(-1, 1) * np.ones((1, self.ny))
        else:
            temps = np.linspace(temp_range[0], temp_range[1], self.ny)
            self.T = np.ones((self.nx, 1)) * temps.reshape(1, -1)
        return self
    
    def add_heat_source(self, location, size, power):
        """Add localized heat source."""
        x, y = location
        self.T[max(0, x-size):min(self.nx, x+size),
               max(0, y-size):min(self.ny, y+size)] += power
        return self
        
    def save_snapshot(self):
        """Store current state in history."""
        self.history.append({
            'time': self.time,
            'temperature': self.T.copy(),
            'max_temp': self.T.max(),
            'total_energy': self.T.sum()
        })
        return self
    
    def plot_current(self, title=None):
        """Plot current temperature distribution."""
        plt.figure(figsize=(8, 6))
        plt.imshow(self.T, cmap='hot', origin='lower')
        plt.colorbar(label='Temperature')
        plt.title(title or f'Temperature at t={self.time:.2f}')
        plt.show()
        return self

class BoundaryCondition(ABC):
    """Abstract base class for boundary conditions."""
    
    @abstractmethod
    def apply(self, T):
        """Apply boundary condition to temperature field."""
        pass

class DirichletBC(BoundaryCondition):
    """Fixed temperature boundary condition."""
    
    def __init__(self, value=0.0):
        self.value = value
        
    def apply(self, T):
        T[0, :] = self.value
        T[-1, :] = self.value
        T[:, 0] = self.value
        T[:, -1] = self.value

class NeumannBC(BoundaryCondition):
    """Zero flux (insulated) boundary condition."""
    
    def apply(self, T):
        T[0, :] = T[1, :]
        T[-1, :] = T[-2, :]
        T[:, 0] = T[:, 1]
        T[:, -1] = T[:, -2]

class MixedBC(BoundaryCondition):
    """Mixed boundary conditions on different edges."""
    
    def __init__(self, top='neumann', bottom='dirichlet', left='neumann', right='dirichlet'):
        self.conditions = {'top': top, 'bottom': bottom, 'left': left, 'right': right}
    
    def apply(self, T):
        # Top boundary
        if self.conditions['top'] == 'dirichlet':
            T[-1, :] = 0
        else:
            T[-1, :] = T[-2, :]
            
        # Bottom boundary  
        if self.conditions['bottom'] == 'dirichlet':
            T[0, :] = 0
        else:
            T[0, :] = T[1, :]
            
        # Left boundary
        if self.conditions['left'] == 'dirichlet':
            T[:, 0] = 0
        else:
            T[:, 0] = T[:, 1]
            
        # Right boundary
        if self.conditions['right'] == 'dirichlet':
            T[:, -1] = 0
        else:
            T[:, -1] = T[:, -2]

class DiffusionSolver:
    """Heat diffusion solver with configurable parameters and methods."""
    
    def __init__(self, grid: HeatGrid, alpha=0.1, dt=0.01):
        self.grid = grid
        self.alpha = alpha
        self.dt = dt
        self.boundary_condition = DirichletBC()  # Default BC
        
    def set_boundary_condition(self, bc: BoundaryCondition):
        """Set boundary condition object."""
        self.boundary_condition = bc
        return self
    
    def step(self):
        """Perform one time step using explicit finite difference."""
        T = self.grid.T
        dx = self.grid.dx
        
        # Compute Laplacian using vectorized operations
        T_new = T.copy()
        laplacian = np.zeros_like(T)
        
        # Interior points
        laplacian[1:-1, 1:-1] = (T[2:, 1:-1] + T[:-2, 1:-1] + 
                                T[1:-1, 2:] + T[1:-1, :-2] - 4*T[1:-1, 1:-1]) / (dx*dx)
        
        # Update interior
        T_new[1:-1, 1:-1] = T[1:-1, 1:-1] + self.alpha * self.dt * laplacian[1:-1, 1:-1]
        
        # Apply boundary conditions
        self.boundary_condition.apply(T_new)
        
        self.grid.T = T_new
        self.grid.time += self.dt
        return self
    
    def solve(self, n_steps, save_every=None):
        """Solve for n_steps with optional history saving."""
        save_interval = save_every if save_every else n_steps // 10
        
        self.grid.save_snapshot()  # Initial condition
        
        for step in range(n_steps):
            self.step()
            
            if step % save_interval == 0:
                self.grid.save_snapshot()
                
        return self
    
    def get_stability_limit(self):
        """Calculate CFL stability limit for explicit scheme."""
        return self.grid.dx**2 / (4 * self.alpha)
    
    def check_stability(self):
        """Check if current dt satisfies stability criterion."""
        dt_max = self.get_stability_limit()
        is_stable = self.dt <= dt_max
        return is_stable, dt_max

In [ ]:
# Level 2: Elegant object-oriented workflow with method chaining

# Example 1: Simple hotspot simulation
grid1 = HeatGrid(40, 40).set_initial_hotspot().save_snapshot()
solver1 = DiffusionSolver(grid1, alpha=0.15, dt=0.005)

# Check stability
stable, dt_max = solver1.check_stability()
print(f"Stability check: {'STABLE' if stable else 'UNSTABLE'} (dt_max = {dt_max:.4f})")

# Solve with method chaining
solver1.solve(200, save_every=40).grid.plot_current("Level 2: Hotspot Evolution")

# Example 2: Complex initial condition with mixed boundaries
grid2 = (HeatGrid(50, 50)
         .set_initial_gradient('x', (0, 50))
         .add_heat_source((10, 25), 3, 100)
         .add_heat_source((40, 25), 3, 80)
         .save_snapshot())

mixed_bc = MixedBC(top='neumann', bottom='dirichlet', left='neumann', right='dirichlet')
solver2 = (DiffusionSolver(grid2, alpha=0.1, dt=0.01)
           .set_boundary_condition(mixed_bc)
           .solve(300, save_every=60))

grid2.plot_current("Level 2: Complex IC with Mixed BC")

# Analyze evolution
times = [snap['time'] for snap in grid2.history]
max_temps = [snap['max_temp'] for snap in grid2.history]
energies = [snap['total_energy'] for snap in grid2.history]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
ax1.plot(times, max_temps, 'r-o', markersize=4)
ax1.set_xlabel('Time')
ax1.set_ylabel('Maximum Temperature')
ax1.set_title('Temperature Evolution')
ax1.grid(True)

ax2.plot(times, energies, 'b-s', markersize=4)
ax2.set_xlabel('Time')
ax2.set_ylabel('Total Energy')
ax2.set_title('Energy Conservation')
ax2.grid(True)

plt.tight_layout()
plt.show()

print(f"Energy conservation: {(energies[-1]/energies[0] - 1)*100:.1f}% change")

**Level 2 Benefits:**
- Clean, readable workflow with method chaining
- Encapsulated state management (grid, time, history)
- Extensible boundary condition system
- Built-in analysis tools and stability checking
- Easy to add new solver methods or grid types

## Level 3: Factory Pattern & Strategy Design

**Characteristics:**
- Abstract interfaces for different algorithms
- Factory methods for object creation
- Strategy pattern for interchangeable components
- Builder pattern for complex configuration

**When to use:** Scientific libraries, multiple solver algorithms, complex parameter spaces

In [ ]:
# Level 3: Factory pattern and strategy design for maximum flexibility

class SolverStrategy(Protocol):
    """Protocol defining the solver interface."""
    def solve_step(self, T: np.ndarray, alpha: float, dt: float, dx: float) -> np.ndarray:
        """Solve one time step."""
        ...

class ExplicitSolver:
    """Explicit finite difference solver."""
    
    def solve_step(self, T: np.ndarray, alpha: float, dt: float, dx: float) -> np.ndarray:
        T_new = T.copy()
        laplacian = np.zeros_like(T)
        laplacian[1:-1, 1:-1] = (T[2:, 1:-1] + T[:-2, 1:-1] + 
                                T[1:-1, 2:] + T[1:-1, :-2] - 4*T[1:-1, 1:-1]) / (dx*dx)
        T_new[1:-1, 1:-1] = T[1:-1, 1:-1] + alpha * dt * laplacian[1:-1, 1:-1]
        return T_new

class ImplicitSolver:
    """Implicit finite difference solver (simplified - would need linear algebra in practice)."""
    
    def solve_step(self, T: np.ndarray, alpha: float, dt: float, dx: float) -> np.ndarray:
        # Simplified implicit step (in reality would solve linear system)
        # Using smaller effective dt for stability demonstration
        effective_dt = dt * 0.5  # Implicit allows larger time steps
        
        T_new = T.copy()
        laplacian = np.zeros_like(T)
        laplacian[1:-1, 1:-1] = (T[2:, 1:-1] + T[:-2, 1:-1] + 
                                T[1:-1, 2:] + T[1:-1, :-2] - 4*T[1:-1, 1:-1]) / (dx*dx)
        T_new[1:-1, 1:-1] = T[1:-1, 1:-1] + alpha * effective_dt * laplacian[1:-1, 1:-1]
        return T_new

class CrankNicolsonSolver:
    """Crank-Nicolson solver (semi-implicit)."""
    
    def solve_step(self, T: np.ndarray, alpha: float, dt: float, dx: float) -> np.ndarray:
        # Simplified C-N (averages explicit and implicit)
        explicit_solver = ExplicitSolver()
        T_explicit = explicit_solver.solve_step(T, alpha, dt*0.5, dx)
        
        # Second half-step
        laplacian = np.zeros_like(T_explicit)
        laplacian[1:-1, 1:-1] = (T_explicit[2:, 1:-1] + T_explicit[:-2, 1:-1] + 
                                T_explicit[1:-1, 2:] + T_explicit[1:-1, :-2] - 
                                4*T_explicit[1:-1, 1:-1]) / (dx*dx)
        T_new = T_explicit.copy()
        T_new[1:-1, 1:-1] = T_explicit[1:-1, 1:-1] + alpha * dt*0.5 * laplacian[1:-1, 1:-1]
        return T_new

class SolverFactory:
    """Factory for creating different solver types."""
    
    _solvers = {
        'explicit': ExplicitSolver,
        'implicit': ImplicitSolver, 
        'crank_nicolson': CrankNicolsonSolver
    }
    
    @classmethod
    def create_solver(cls, solver_type: str) -> SolverStrategy:
        """Create solver instance by name."""
        if solver_type not in cls._solvers:
            available = ', '.join(cls._solvers.keys())
            raise ValueError(f"Unknown solver '{solver_type}'. Available: {available}")
        return cls._solvers[solver_type]()
    
    @classmethod
    def list_solvers(cls):
        """List available solver types."""
        return list(cls._solvers.keys())

class BoundaryConditionFactory:
    """Factory for creating boundary conditions."""
    
    @staticmethod
    def create_dirichlet(value=0.0):
        return DirichletBC(value)
    
    @staticmethod  
    def create_neumann():
        return NeumannBC()
    
    @staticmethod
    def create_mixed(**kwargs):
        return MixedBC(**kwargs)
    
    @staticmethod
    def from_config(config: Dict[str, Any]):
        """Create BC from configuration dictionary."""
        bc_type = config.get('type', 'dirichlet')
        
        if bc_type == 'dirichlet':
            return DirichletBC(config.get('value', 0.0))
        elif bc_type == 'neumann':
            return NeumannBC()
        elif bc_type == 'mixed':
            return MixedBC(**config.get('conditions', {}))
        else:
            raise ValueError(f"Unknown BC type: {bc_type}")

class SimulationBuilder:
    """Builder pattern for creating complex simulations."""
    
    def __init__(self):
        self._config = {
            'grid_size': (50, 50),
            'dx': 1.0,
            'alpha': 0.1,
            'dt': 0.01,
            'solver_type': 'explicit',
            'boundary_config': {'type': 'dirichlet', 'value': 0.0},
            'initial_condition': 'hotspot'
        }
    
    def grid_size(self, nx, ny):
        self._config['grid_size'] = (nx, ny)
        return self
    
    def spatial_step(self, dx):
        self._config['dx'] = dx
        return self
        
    def thermal_diffusivity(self, alpha):
        self._config['alpha'] = alpha
        return self
    
    def time_step(self, dt):
        self._config['dt'] = dt
        return self
    
    def solver(self, solver_type):
        self._config['solver_type'] = solver_type
        return self
    
    def boundary_conditions(self, **bc_config):
        self._config['boundary_config'] = bc_config
        return self
    
    def initial_condition(self, condition_type):
        self._config['initial_condition'] = condition_type
        return self
    
    def build(self):
        """Build and return configured simulation."""
        # Create grid
        nx, ny = self._config['grid_size']
        grid = HeatGrid(nx, ny, self._config['dx'])
        
        # Set initial condition
        if self._config['initial_condition'] == 'hotspot':
            grid.set_initial_hotspot()
        elif self._config['initial_condition'] == 'gradient':
            grid.set_initial_gradient()
            
        # Create solver
        solver_strategy = SolverFactory.create_solver(self._config['solver_type'])
        boundary_condition = BoundaryConditionFactory.from_config(self._config['boundary_config'])
        
        return AdvancedDiffusionSolver(grid, solver_strategy, boundary_condition,
                                     self._config['alpha'], self._config['dt'])

class AdvancedDiffusionSolver:
    """Advanced solver using strategy pattern."""
    
    def __init__(self, grid: HeatGrid, solver_strategy: SolverStrategy, 
                 boundary_condition: BoundaryCondition, alpha: float, dt: float):
        self.grid = grid
        self.solver_strategy = solver_strategy
        self.boundary_condition = boundary_condition
        self.alpha = alpha
        self.dt = dt
        
    def step(self):
        """Perform one time step using the configured strategy."""
        self.grid.T = self.solver_strategy.solve_step(self.grid.T, self.alpha, 
                                                     self.dt, self.grid.dx)
        self.boundary_condition.apply(self.grid.T)
        self.grid.time += self.dt
        return self
    
    def solve(self, n_steps, save_every=None):
        """Solve with progress tracking."""
        save_interval = save_every or n_steps // 10
        self.grid.save_snapshot()
        
        for step in range(n_steps):
            self.step()
            if step % save_interval == 0:
                self.grid.save_snapshot()
        return self

In [ ]:
# Level 3: Factory and builder patterns enable powerful workflows

print("Available solvers:", SolverFactory.list_solvers())

# Solver comparison using factory pattern
solvers_to_test = ['explicit', 'crank_nicolson', 'implicit']
results = {}

for solver_name in solvers_to_test:
    # Build simulation with fluent interface
    simulation = (SimulationBuilder()
                  .grid_size(30, 30)
                  .thermal_diffusivity(0.1)
                  .time_step(0.015)  # Larger time step to test stability
                  .solver(solver_name)
                  .initial_condition('hotspot')
                  .boundary_conditions(type='neumann')
                  .build())
    
    # Time the simulation
    start_time = time.time()
    simulation.solve(100, save_every=25)
    elapsed = time.time() - start_time
    
    results[solver_name] = {
        'time': elapsed,
        'final_temp': simulation.grid.T.max(),
        'grid': simulation.grid
    }

# Compare results
print("\nSolver Performance Comparison:")
print("-" * 50)
for name, result in results.items():
    print(f"{name:15}: {result['time']:.4f}s, max_temp={result['final_temp']:.2f}")

# Visualize comparison
fig, axes = plt.subplots(1, len(solvers_to_test), figsize=(4*len(solvers_to_test), 3))
vmax = max(result['final_temp'] for result in results.values())

for i, (name, result) in enumerate(results.items()):
    im = axes[i].imshow(result['grid'].T, cmap='hot', origin='lower', vmin=0, vmax=vmax)
    axes[i].set_title(f'{name.replace("_", " ").title()}\nTime: {result["time"]:.3f}s')
    axes[i].axis('off')

plt.colorbar(im, ax=axes, fraction=0.02, pad=0.04)
plt.suptitle('Level 3: Solver Strategy Comparison', y=1.02)
plt.tight_layout()
plt.show()

# Advanced configuration example
complex_simulation = (SimulationBuilder()
                     .grid_size(60, 40)
                     .thermal_diffusivity(0.15)
                     .time_step(0.008)
                     .solver('explicit')
                     .boundary_conditions(type='mixed', 
                                        top='neumann', bottom='dirichlet',
                                        left='neumann', right='dirichlet')
                     .initial_condition('gradient')
                     .build())

complex_simulation.solve(250, save_every=50).grid.plot_current("Level 3: Complex Configuration")

print(f"\nSimulation completed: {len(complex_simulation.grid.history)} snapshots saved")

## Summary: When to Use Each Abstraction Level

| Level | Best For | Pros | Cons |
|-------|----------|------|------|
| **Level 0: Direct** | Quick prototypes, equation exploration | Fast to write, easy to understand | No reusability, hard to maintain |
| **Level 1: Functions** | Research code, parameter studies | Reusable, testable, flexible | Can become complex with many parameters |
| **Level 2: Classes** | Complex simulations, GUI apps | Clean API, state management, extensible | More code, learning curve |
| **Level 3: Patterns** | Scientific libraries, frameworks | Maximum flexibility, professional quality | Complex architecture, over-engineering risk |

### Scientific Computing Guidelines:

1. **Start simple** - Begin with Level 0/1 for exploration
2. **Refactor when needed** - Move to higher levels as complexity grows
3. **Consider your audience** - Researchers may prefer functions, developers prefer classes
4. **Balance abstraction vs. transparency** - Scientific code needs to be verifiable
5. **Performance matters** - Don't sacrifice speed for elegance in compute-intensive code

Each level serves its purpose in the scientific computing workflow!